In [1]:
import pandas as pd
import codecs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [2]:
source = [(r'data/Train_data/Nonsports/', 'NONSPORTS'),
          (r'data/Train_data/Sports/', 'SPORTS')
          ]

In [3]:
def build_data_frame(fpath, label):
    rows = []
    for file_name in os.listdir(fpath):
        path = os.path.join(fpath,file_name)
        rows.append({'text': path, 'labels': label })
    data_frame = pd.DataFrame(rows)
    return(data_frame)

In [4]:
# def clean_data(data):
#     # Remove punctuations, digits and stop words
#     # Removal of repeated contents
#     valid_text=[]
#     stop = set(stopwords.words('english'))

#     for text in data:
#         words=re.findall(r'\b[a-zA-Z][a-z]{2,9}\b',text)
#         valid_words=[]
#         for word in words:
#             if word not in stop:
#                 valid_words.append(word)
#         valid_text.append(' '.join(valid_words))
#     return valid_text

In [5]:
data = pd.DataFrame({'text': [], 'labels':[]})

In [6]:
for fpath, label in source:
    data = data.append(build_data_frame(fpath, label))
#print(data)


C:\Users\nirale1.kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
train_txt = [codecs.open(file,encoding='iso 8859-1').read() for file in data['text'].values]

In [8]:
train_txt[1]

'Part III will be the final post of this series, where I will describe advanced data engineering patterns, higher level abstractions, and extended frameworks that would make building ETLs a lot easier and more efficient. A lot of these patterns are taught to me by Airbnbâ\x80\x99s experienced data engineers who learned the hard way. I find these insights particularly useful for seasoned data scientists and seasoned data engineers who are trying to further optimize their workflows.\r\nGiven that I am now a huge proponent for learning data engineering as an adjacent discipline, you might find it surprising that I had the completely opposite opinion a few years agoâ\x80\x8aâ\x80\x94â\x80\x8aI struggled a lot with data engineering during my first job, both motivationally and emotionally.\r\n\r\n \r\n\r\nMy First Industry Job out of Graduate School\r\n \r\nRight after graduate school, I was hired as the first data scientist at a small startup affiliated with the Washington Post. With endles

In [9]:
final_data = pd.DataFrame({'Text':train_txt, 'labels':data['labels'].values})

In [10]:
final_data.head()

,Text,labels
0,The Indian economy appears to have absorbed th...,NONSPORTS
1,Part III will be the final post of this series...,NONSPORTS
2,"Reflecting on this experience, I realized that...",NONSPORTS
3,"Unfortunately, many companies do not realize t...",NONSPORTS
4,The Indian economy appears to have absorbed th...,NONSPORTS


In [11]:
final_data.shape

(40, 2)

# Cleansing the data
#Remove stop words,punctuations,non-words,stemming,lemmatizing

In [12]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
my_stop=set(stopwords.words('english')+list(punctuation))

In [110]:
def split_into_lemmas(message):
    message=message.lower()
    words = word_tokenize(message)
    words_sans_stop=[]
    for word in words :
        if word in my_stop:continue
        words_sans_stop.append(word)
    return [lemma.lemmatize(word) for word in words_sans_stop]

# Splitting the training and testing data. (Preparing for vectorizer)

In [20]:
mydata_train,mydata_test=train_test_split(final_data,test_size=0.2,
                                          random_state=2)

In [21]:
mydata_train.shape

(32, 2)

In [22]:
mydata_test.shape

(8, 2)

In [23]:
mydata_train.reset_index(drop=True,inplace=True)


In [24]:
mydata_test.shape

(8, 2)

In [44]:
final_data['labels'].value_counts()

SPORTS       20
NONSPORTS    20
Name: labels, dtype: int64

In [25]:
tf= CountVectorizer(analyzer=split_into_lemmas,min_df=10,max_df=500,
                       stop_words=my_stop)

In [151]:
tf

CountVectorizer(analyzer=<function split_into_lemmas at 0x000002BDDB7C6598>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=500,
        max_features=None, min_df=10, ngram_range=(1, 1),
        preprocessor=None,
        stop_words={'i', ')', '[', "she's", "won't", 'themselves', 'up', 'such', "isn't", 'how', 'a', 'hasn', ';', 'ain', "you're", 'as', ']', '#', 'mightn', 'being', 'with', 've', 'y', 'doesn', '!', 'were', 'weren', 'll', 'they', 'doing', '(', 'yours', 'and', 'needn', 'was', 'only', "didn't", "shouldn't", ...nce', 'if', 'very', 'so', 'your', 'whom', '"', 'mustn', 'has', '}', 'when', "that'll", 'what', 'am'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [45]:
tf.fit(mydata_train['Text'])

train_tf=tf.transform(mydata_train['Text'])


x_train_tf = pd.DataFrame(train_tf.toarray(), 
                   columns=tf.get_feature_names())

test_tf=tf.transform(mydata_test['Text'])

x_test_tf=pd.DataFrame(test_tf.toarray(),
                       columns=tf.get_feature_names())

In [47]:
y_train=mydata_train['labels']
y_test=mydata_test['labels']

In [48]:
y_test.shape

(8,)

In [49]:
#x_train_tf.head()

In [50]:
from sklearn.preprocessing import StandardScaler

In [51]:
st = StandardScaler()

In [52]:
st.fit(x_train_tf)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [53]:
x_train_tf_scaled=st.transform(x_train_tf)

In [54]:
x_test_tf_scaled=st.transform(x_test_tf)

In [55]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [56]:
knn=KNeighborsClassifier(n_neighbors=10)

In [57]:
knn.fit(x_train_tf_scaled,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [58]:
predictions=knn.predict(x_test_tf_scaled)

In [59]:
accuracy_score(y_test,predictions)

0.875

In [60]:
clf_svm=SVC()

In [61]:
clf_svm.fit(x_train_tf_scaled,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [62]:
accuracy_score(y_test,clf_svm.predict(x_test_tf_scaled))

0.875

# predicting on new set of test data 

In [68]:
test_data_folder = 'data/Test_data/'

In [73]:
import glob

In [106]:
test = []
for file in glob.glob(test_data_folder+"*.txt"):
    test.append(codecs.open(file,encoding='iso 8859-1').read())


In [116]:
# test_data_clean = clean_data(test)

In [115]:
# def clean_data(data):
#     # Remove punctuations, digits and stop words
#     # Removal of repeated contents
#     valid_text=[]
#     stop = set(stopwords.words('english'))

#     for text in data:
#         words=re.findall(r'\b[a-zA-Z][a-z]{2,9}\b',text)
#         valid_words=[]
#         for word in words:
#             if word not in stop:
#                 valid_words.append(word)
#         valid_text.append(' '.join(valid_words))
#     return valid_text

# Preparing test data 

In [149]:
tf_test= CountVectorizer(analyzer=split_into_lemmas,min_df=1,max_df=500,
                       stop_words=my_stop)

In [152]:
tf_test.fit(test)

CountVectorizer(analyzer=<function split_into_lemmas at 0x000002BDDB7C6598>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=500,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,
        stop_words={'i', ')', '[', "she's", "won't", 'themselves', 'up', 'such', "isn't", 'how', 'a', 'hasn', ';', 'ain', "you're", 'as', ']', '#', 'mightn', 'being', 'with', 've', 'y', 'doesn', '!', 'were', 'weren', 'll', 'they', 'doing', '(', 'yours', 'and', 'needn', 'was', 'only', "didn't", "shouldn't", ...nce', 'if', 'very', 'so', 'your', 'whom', '"', 'mustn', 'has', '}', 'when', "that'll", 'what', 'am'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [153]:
test_tf1=tf_test.transform(test)



In [154]:
x_test_tf1=pd.DataFrame(test_tf1.toarray(),
                       columns=tf_test.get_feature_names())

In [156]:
st.fit(x_test_tf1)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [157]:
x_test_tf_scaled1=st.transform(x_test_tf1)

In [161]:
x_test_tf_scaled1.shape

(1, 4886)

In [159]:
x_test_tf_scaled1 = x_test_tf_scaled1.reshape(1,-1)

In [167]:
knn.predict(x_test_tf_scaled1)

ValueError: query data dimension must match training data dimension